# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [47]:
# Use this cell to write your code for Task 1
import pandas as pd
house_sales = pd.read_csv('house_sales.csv')

missing_city = house_sales['city'].isna().sum() + (house_sales['city'] == '--').sum()


print('Unique values in City columnL: ', house_sales['city'].unique())
print(f'Number of missing values in the city column: {missing_city}')

Unique values in City columnL:  ['Silvertown' 'Riverford' 'Teasdale' 'Poppleton' '--']
Number of missing values in the city column: 73


In [43]:
house_sales['city'].unique()

Number of missing values in the city column: 73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [8]:
# Use this cell to write your code for Task 2
clean_data = pd.read_csv('house_sales.csv')

clean_data = clean_data.dropna(subset=['house_id'])


clean_data['city'].fillna('Unknown', inplace=True)
clean_data['city'].replace('--','Unknown', inplace=True)
print('Unique values city: ',clean_data['city'].unique())
print('Amount of missing values in city',clean_data['city'].isnull().sum())


clean_data = clean_data[clean_data['sale_price'].notnull()]
print('Amount of missing values in sale_price: ',
      clean_data['sale_price'].isnull().sum())

clean_data['sale_date'].fillna('2023-01-01', inplace=True)
print('Amount of missing values in sale_date',
      clean_data['sale_date'].isnull().sum())


clean_data['months_listed'].fillna(round(clean_data['months_listed'].mean(), 1), inplace=True)
print('Amount of missing values in month_listed',
      clean_data['months_listed'].isnull().sum())


clean_data['bedrooms'].fillna(round(clean_data['bedrooms'].mean(), 1), inplace=True)
print('Amount of missing values in bedrooms',
      clean_data['bedrooms'].isnull().sum())


print('Unique values inn house_type: ',clean_data['house_type'].unique())
clean_data['house_type'] = clean_data['house_type'].replace({
    'Terr.':'Terraced',
    'Semi' : 'Semi-detached',
    'Det.': 'Detached'

})
clean_data['house_type'].fillna(clean_data['house_type'].mode()[0], inplace=True)
print('Amount of missing values in house_type',
      clean_data['house_type'].isnull().sum())

print('Dat type  Area: ',clean_data['area'].dtype)
clean_data['area'] = clean_data['area'].str.replace(
    ' sq.m.', '', regex=False).astype(float)

clean_data['area'].fillna(round(clean_data['area'].mean(), 1), inplace=True)

['Silvertown' 'Riverford' 'Teasdale' 'Poppleton' 'Unknown']
0
0
0
0
0
['Semi-detached' 'Detached' 'Det.' 'Terraced' 'Semi' 'Terr.']
0
Type column Area:  object


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [17]:
# Use this cell to write your code for Task 3
price_by_rooms = pd.read_csv('house_sales.csv')

price_by_rooms = price_by_rooms.groupby('bedrooms')['sale_price'].agg(
  avg_price ='mean',
  var_price ='var'
).reset_index()

price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)


price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [30]:
# # Use this cell to write your code for Task 4
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression


# train = pd.read_csv('train.csv')
# validation = pd.read_csv('validation.csv')

# features = train[['city', 'months_listed', 'bedrooms', 'house_type', 'area']]
# target = train['sale_price']

# X_train = train[features]
# y_train = validation[target]

# model = LinearRegression()
# model.fit(X_train, y_train)

# X_validation = validation[features]
# predict = model.predict(X_validation)

# base_result = pd.DataFrame({
#   'house_id': validation['house_id'],
#   'price': predict
# })

# print(base_result)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('train.csv')
validation_df = pd.read_csv('validation.csv')

features = ['house_type', 'bedrooms', 'area', 'months_listed', 'city']

label_encoders = {}
for col in ['house_type', 'city']:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    validation_df[col] = le.fit_transform(validation_df[col].astype(str))


X_train = train_df[features]
y_train = train_df['sale_price']

model = LinearRegression()
model.fit(X_train, y_train)

X_validation = validation_df[features]
predictions = model.predict(X_validation)

base_result = pd.DataFrame({
    'house_id': validation_df['house_id'],
    'price': predictions
})

base_result

,house_id,price
0,1331375,114410.640068
1,1630115,298257.275635
2,1645745,385953.871714
3,1336775,122798.036772
4,1888274,272134.780558
...,...,...
295,1986255,352195.395262
296,1896276,371554.686616
297,1758223,260839.885628
298,1752010,155222.782542


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [37]:
# Use this cell to write your code for Task 5
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('train.csv')
validation = pd.read_csv('validation.csv')

features = ['city', 'months_listed', 'bedrooms', 'house_type', 'area']

label = {}
for col in ['house_type', 'city']:
  label_encoder = LabelEncoder()
  train[col] = label_encoder.fit_transform(train[col].astype(str))
  validation[col] = label_encoder.fit_transform(validation[col].astype(str))

X_train = train[features]
y_train = train['sale_price']

model = RandomForestRegressor()
model.fit(X_train, y_train)

X_validation = validation[features]
predict = model.predict(X_validation)

compare_result = pd.DataFrame({
  'house_id':validation['house_id'],
  'price' : predict
})

compare_result

,house_id,price
0,1331375,82584.440000
1,1630115,302558.550000
2,1645745,407199.530000
3,1336775,106326.380000
4,1888274,267082.410000
...,...,...
295,1986255,359774.693333
296,1896276,398016.290000
297,1758223,262521.880000
298,1752010,179096.120000
